In [1]:
#%pip install ultralytics

In [2]:
import ultralytics
ultralytics.__version__

'8.2.14'

In [3]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')

In [4]:
class_list = ['person', 'bicycle', 'car', 'motorbike', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [5]:
tracker=Tracker()
count=0

In [6]:
import os
import time
import cv2
import pandas as pd

# Initialize the video capture
cap = cv2.VideoCapture(r'C:\Users\HP\Downloads\TransitCount\video-01_1LUhNCQe.mp4')

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Initialize counters and storage for vehicle tracking
down = {}
up = {}
counter_down = []
counter_up = []

counter_down_cars = []
counter_down_motorbikes = []
counter_down_trucks = []
counter_up_cars = []
counter_up_motorbikes = []
counter_up_trucks = []

# Line positions
red_line_y = 198
blue_line_y = 268
offset = 6

# Create a folder to save frames
if not os.path.exists('detected_frames'):
    os.makedirs('detected_frames')

# Video writer setup
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 60.0, (1020, 500))

# User choice for vehicle type to display
vehicle_type_choice = input("Enter vehicle type to watch (cars, motorbikes, trucks, all): ").strip().lower()

# Load your object detection model here
# model = ... (Load your model)

# Initialize your tracker here
# tracker = ... (Initialize your tracker)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    frame = cv2.resize(frame, (1020, 500))

    # Model prediction
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")
    detections = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        if d in [1, 2, 3, 7]:  # Check for car (ID 2), motorbike (ID 3), bicycle (ID 1), or truck (ID 7)
            detections.append([x1, y1, x2, y2, d])
    
    bbox_list = [[x1, y1, x2, y2] for x1, y1, x2, y2, d in detections]
    bbox_id = tracker.update(bbox_list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)

        vehicle_class = None
        for detection in detections:
            if detection[0] == x3 and detection[1] == y3 and detection[2] == x4 and detection[3] == y4:
                vehicle_class = detection[4]
                break

        if vehicle_class is None:
            continue

        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = (time.time(), vehicle_class)

        if id in down:
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                if counter_down.count(id) == 0:
                    counter_down.append(id)
                    if vehicle_class == 2:
                        counter_down_cars.append(id)
                    elif vehicle_class == 3:
                        counter_down_motorbikes.append(id)
                    elif vehicle_class == 7:
                        counter_down_trucks.append(id)
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)

        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
            up[id] = (time.time(), vehicle_class)

        if id in up:
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                if counter_up.count(id) == 0:
                    counter_up.append(id)
                    if vehicle_class == 2:
                        counter_up_cars.append(id)
                    elif vehicle_class == 3:
                        counter_up_motorbikes.append(id)
                    elif vehicle_class == 7:
                        counter_up_trucks.append(id)
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)

    text_color = (0, 0, 0)  # Black color for text
    yellow_color = (0, 255, 255)  # Yellow color for background
    red_color = (0, 0, 255)  # Red color for lines

    cv2.rectangle(frame, (670, 0), (1020, 270), yellow_color, -1)

    cv2.line(frame, (100, 198), (774, 198), red_color, 2)
    cv2.putText(frame, ('Line'), (100, 198), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.line(frame, (8, 268), (927, 268), red_color, 2)
    cv2.putText(frame, ('Line'), (8, 268), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    if vehicle_type_choice == 'cars' or vehicle_type_choice == 'all':
        cv2.putText(frame, ('Cars Entering - ' + str(len(counter_down_cars))), (680, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        cv2.putText(frame, ('Cars Exiting - ' + str(len(counter_up_cars))), (680, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    if vehicle_type_choice == 'motorbikes' or vehicle_type_choice == 'all':
        cv2.putText(frame, ('Motorbikes Entering - ' + str(len(counter_down_motorbikes))), (680, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        cv2.putText(frame, ('Motorbikes Exiting - ' + str(len(counter_up_motorbikes))), (680, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    if vehicle_type_choice == 'trucks' or vehicle_type_choice == 'all':
        cv2.putText(frame, ('Trucks Entering - ' + str(len(counter_down_trucks))), (680, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        cv2.putText(frame, ('Trucks Exiting - ' + str(len(counter_up_trucks))), (680, 240), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    if vehicle_type_choice == 'all':
        cv2.putText(frame, ('ENTERING LINE - ' + str(len(counter_down))), (680, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
        cv2.putText(frame, ('EXIT LINE - ' + str(len(counter_up))), (680, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

        total_vehicles = len(counter_down) + len(counter_up)
        cv2.putText(frame, ('TOTAL VEHICLES - ' + str(total_vehicles)), (680, 270), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    frame_filename = f'detected_frames/frame_{int(time.time())}.jpg'
    cv2.imwrite(frame_filename, frame)

    out.write(frame)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 320x640 (no detections), 215.1ms
Speed: 6.0ms preprocess, 215.1ms inference, 1229.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 156.2ms
Speed: 3.1ms preprocess, 156.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 146.9ms
Speed: 2.0ms preprocess, 146.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 149.0ms
Speed: 2.0ms preprocess, 149.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 140.0ms
Speed: 2.0ms preprocess, 140.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 (no detections), 156.7ms
Speed: 1.0ms preprocess, 156.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 149.1ms
Speed: 2.0ms preprocess, 149.1ms inference, 48.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 146.8ms
Speed: 2.0ms preprocess, 146.8ms 

In [7]:
#!pip install pylance

In [8]:
%pip install yolov8

Note: you may need to restart the kernel to use updated packages.


In [9]:
#%pip install yolov8s.pt